In [1]:
import os
import json
from langchain_ollama import ChatOllama

In [9]:
llm = ChatOllama(
    model="llama3.1",
    temperature=0.5,
    # other params...
)

In [ ]:
# TODO: add few shot examples and implement it
# https://python.langchain.com/v0.1/docs/modules/model_io/prompts/few_shot_examples/
examples = [
    {
        'question': '{"song": "Embraced", "artist": "Paul Cardall", "instrument": "Organ", "genre": "Christian Relaxative", "tempo": 60, "key": "C#", "mode": "Major", "energy": 0.0398, "danceability": 0.346, "pedals": [{"name": "Phaser", "rate_hz": 7.584981514363318, "depth": 0.04263510728352049, "centre_frequency_hz": 698.3204252362798}]}',
        'answer': 'Mellow Christian organ music in C# major at 60 BPM with subtle phaser effect'
    }
]

In [ ]:
jsons = [f for f in os.listdir('renders') if f.endswith('.json')]
for json_file in jsons:
    f = open(f'renders/{json_file}', encoding='utf-8')
    data = json.load(f)
    messages = [
    (
        "system",
        "You are a helpful assistant that turns a .json file into a human-written-like prompt for a music generation AI model. The prompt should be as short as possible and very straightforward. It should be a description of the desired sound, not a petition, so avoid verbs. Reply only with the prompt, no extra text or quotes. Ignore song title and artist unless they're extremely popular. State the exact tempo (BPM)",
    ),
    (
        "human",
        str(data)
    )
    ]
    ai_msg = llm.invoke(messages)
    print(ai_msg.content)
    # Add prompt to .json
    # data['prompt'] = ai_msg.content
    # with open(f'renders/{json_file}', 'w') as json_file:
    #     json.dump(data, json_file)
